In [5]:
import pandas as pd
import numpy as np

# === Étape 1 : Chargement du fichier Excel ===
file_path = "2025-04-28_EC scenarios data_Decomposition.xlsx"

# === Étape 2 : Configuration des onglets et scénarios ===
sheet_names = ["Buildings -Services", "Industry", "PassLandTransport"]

scenario_labels = {
    "Buildings -Services": ["scenario 1", "scenario 2", "scenario 3", "life scenario"],
    "Industry": [
        "scenario 1 -standard",
        "scenario 2-standard",
        "scenario 3-standard",
        "life scenario-circular economy"
    ],
    "PassLandTransport": ["scenario 1", "scenario 2", "scenario 3", "life scenario"]
}

# === Étape 3 : Fonction de décomposition LMDI sécurisée ===
def lmdi_contribution_safe(co2_0, co2_t, x_0, x_t):
    if co2_0 == co2_t or x_0 == 0 or x_t == 0:
        return 0
    weight = (co2_t - co2_0) / (np.log(np.abs(co2_t)) - np.log(np.abs(co2_0)))
    return weight * np.log(np.abs(x_t) / np.abs(x_0))

# === Étape 4 : Traitement de toutes les feuilles et scénarios ===
all_results = {}

for sheet in sheet_names:
    df_sheet = pd.read_excel(file_path, sheet_name=sheet, header=None)
    results = {}
    for scenario in scenario_labels[sheet]:
        # Recherche de la ligne de début du bloc scénario
        idx = df_sheet[df_sheet[0].astype(str).str.strip().str.lower() == scenario.lower()].index[0]
        df_s = df_sheet.iloc[idx + 2 : idx + 5, [0, 1, 2, 3, 4]].copy()
        df_s.columns = ["Year", "Pop", "m2cap", "FE_m2", "CO2e_FE"]
        df_s.set_index("Year", inplace=True)
        df_s.index = df_s.index.astype(int)
        df_s = df_s.astype(float)

        # Calcul du CO2 total physique
        df_s["CO2_total"] = df_s["Pop"] * df_s["m2cap"] * df_s["FE_m2"] * df_s["CO2e_FE"]

        # Préparation des périodes
        co2_2015 = df_s.loc[2015, "CO2_total"]
        co2_2040 = df_s.loc[2040, "CO2_total"]
        co2_2050 = df_s.loc[2050, "CO2_total"]

        contrib_2040_2015 = {}
        contrib_2050_2040 = {}

        for lev in ["Pop", "m2cap", "FE_m2", "CO2e_FE"]:
            x0, xt = df_s.loc[2015, lev], df_s.loc[2040, lev]
            contrib_2040_2015[lev] = abs(lmdi_contribution_safe(co2_2015, co2_2040, x0, xt))

            x0b, xtb = df_s.loc[2040, lev], df_s.loc[2050, lev]
            contrib_2050_2040[lev] = abs(lmdi_contribution_safe(co2_2040, co2_2050, x0b, xtb))

        # Nettoyage du nom du scénario pour affichage
        scenario_clean = scenario.replace("-standard", "").replace("-circular economy", "").title()

        df_result = pd.DataFrame({
            "∆CO2 (2040–2015, abs)": contrib_2040_2015,
            "∆CO2 (2050–2040, abs)": contrib_2050_2040
        }).round(2)

        df_result.loc["Somme leviers (abs)"] = [
            round(sum(contrib_2040_2015.values()), 2),
            round(sum(contrib_2050_2040.values()), 2)
        ]

        results[scenario_clean] = df_result

    all_results[sheet] = pd.concat(results, names=["Scenario", "Levier"])

# === Étape 5 : Regrouper tous les résultats ===
df_final = pd.concat(all_results, names=["Secteur", "Scenario", "Levier"])

# === Étape 6 : Affichage lisible
pd.set_option("display.float_format", '{:,.0f}'.format)
print(df_final)

# === Étape 7 : Export Excel
df_final.to_excel("LMDI_decomposition_autres_secteurs.xlsx")


                                                       ∆CO2 (2040–2015, abs)  \
Secteur             Scenario      Levier                                       
Buildings -Services Scenario 1    Pop                            454,853,358   
                                  m2cap                       19,248,324,415   
                                  FE_m2                       33,269,897,500   
                                  CO2e_FE                    203,501,684,127   
                                  Somme leviers (abs)        256,474,759,400   
                    Scenario 2    Pop                            339,254,757   
                                  m2cap                       14,356,463,469   
                                  FE_m2                       27,126,681,354   
                                  CO2e_FE                    222,540,610,195   
                                  Somme leviers (abs)        264,363,009,774   
                    Scenario 3    Pop   